In [7]:
import os
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO
from tqdm import tqdm

import warnings

warnings.filterwarnings( 'ignore' )

In [6]:
sub_dir = '../../submission'
submission_files = [
                    'result_swin_f0_default_1e-3.csv',
                    'result_swin_f1_default_1e-3.csv',
                    # 'result_swin_f2_default_1e-3.csv',
                    'result_swin_f3_default_1e-3.csv',
                    # 'result_swin_f4_default_1e-3.csv',

                    'result_swin_f0_pseudo_1e-3.csv',
                    'result_swin_f1_pseudo_1e-3.csv',
                    # 'result_swin_f2_pseudo_1e-3.csv',
                    'result_swin_f3_pseudo_1e-3.csv',
                    'result_swin_f4_pseudo_1e-3.csv',

                    'result_yolov5l6_f0_default_1e-3.csv',
                    'result_yolov5l6_f1_default_1e-3.csv',
                    'result_yolov5l6_f2_default_1e-3.csv',
                    'result_yolov5l6_f3_default_1e-3.csv',
                    'result_yolov5l6_f4_default_1e-3.csv',

                    'result_yolov5l6_f2_pseudo_1e-3.csv',
                    'result_yolov5l6_f3_pseudo_1e-3.csv',
                    'result_yolov5l6_f4_pseudo_1e-3.csv',

                    'result_yolov7w6_f0_default_1e-3.csv',
                    'result_yolov7w6_f1_default_1e-3.csv',
                    'result_yolov7w6_f2_default_1e-3.csv',
                    'result_yolov7w6_f3_default_1e-3.csv',
                    'result_yolov7w6_f4_default_1e-3.csv',

                    'result_yolov7w6_f2_pseudo_1e-3.csv',
                    'result_yolov7w6_f3_pseudo_1e-3.csv',
                    'result_yolov7w6_f4_pseudo_1e-3.csv',
                    ]

submission_df = [pd.read_csv(os.path.join(sub_dir, file)) for file in tqdm(submission_files)]

100%|██████████| 22/22 [00:06<00:00,  3.56it/s]


In [23]:
image_ids = submission_df[0]['image_id'].tolist()

In [24]:

# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = './dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [25]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.6

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(tqdm(image_ids)):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:66: UserWarning: Zero area box skipped: [0.0, 0.006640791893005371, 0.0, 0.7060057520866394].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:66: UserWarning: Zero area box skipped: [0.0, 0.009719491004943848, 0.0, 0.7207220196723938].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:66: UserWarning: Zero area box skipped: [0.0, 0.003071203827857971, 0.0, 0.3445202708244324].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:66: UserWarning: Zero area box skipped: [0.0, 0.4234476089477539, 0.0, 0.899232029914856].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/

In [26]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('result_yolov5_yolov7_swin_all_wopseudo_wbfensemble.csv', index = None)

submission.head()

,PredictionString,image_id
0,7 0.8889930248260498 216.75791931152344 51.973...,test/0000.jpg
1,5 0.6938202381134033 132.2281494140625 0.22078...,test/0001.jpg
2,1 0.8105384707450867 76.7972183227539 272.6375...,test/0002.jpg
3,9 0.7210465669631958 144.7139434814453 259.471...,test/0003.jpg
4,1 0.6025466124216715 195.54220581054688 255.07...,test/0004.jpg
